In [1]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('../traces/matmul', header=None)
data.columns = ['branch_id', 'taken']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838802 entries, 0 to 838801
Data columns (total 2 columns):
branch_id    838802 non-null int64
taken        838802 non-null int64
dtypes: int64(2)
memory usage: 12.8 MB


In [49]:
data.iloc[0].taken == False

False

In [30]:
a = np.zeros(2**4, dtype=np.uint16)
a[4]

0

## 8192 Entry BHT (13-bit)

In [36]:
class BHT:
    def __init__(self, bits=13):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.bool)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = value
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]

In [41]:
x = BHT()
print(x.get(6))
x.update(6, 0)
print(x.get(6))

False
False


In [55]:
bht = BHT()
total_predictions, mispredictions = (data.shape[0],0)

for i in range(data.shape[0]):
    branch_id = data.iloc[i].branch_id
    taken = data.iloc[i].taken
    if bht.get(branch_id) != taken:
        mispredictions = mispredictions + 1
        bht.update(branch_id,taken)
        
print(total_predictions, mispredictions)
print(mispredictions/total_predictions*100)

838802 140128
16.70573031537836


In [103]:
class BHT_2:
    def __init__(self, bits=12):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = self.states[self.table[self.key(branch_id)], value]
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]//2

In [114]:
bht = BHT_2()
iters = data.shape[0]
total_predictions, mispredictions = (iters,0)#(data.shape[0],0)

for i in range(iters):
    branch_id = data.iloc[i].branch_id
    taken = data.iloc[i].taken
    if bht.get(branch_id) != taken:
        mispredictions = mispredictions + 1
    bht.update(branch_id,taken)
        
# print(total_predictions, mispredictions)
print(mispredictions/total_predictions*100)

15.868822439622225


In [147]:
class BHT_2_2:
    def __init__(self, bits=10):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros([2**self.bits, 4], dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
        self.global_history = np.zeros(1, dtype=np.uint8)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id), self.global_history[0]] = self.states[self.table[self.key(branch_id), self.global_history[0]], value]
        self.global_history[0] = (self.global_history << 1 | value) % 4
    
    def get(self, branch_id):
        return self.table[self.key(branch_id), self.global_history[0]]//2

In [149]:
bht = BHT_2_2()
iters = data.shape[0]
total_predictions, mispredictions = (iters,0)#(data.shape[0],0)

for i in range(iters):
    branch_id = data.iloc[i].branch_id
    taken = data.iloc[i].taken
    if bht.get(branch_id) != taken:
        mispredictions = mispredictions + 1
    bht.update(branch_id,taken)
        
# print(total_predictions, mispredictions)
print(mispredictions/total_predictions*100)

14.709430831113899


In [131]:
a = np.zeros([4,2])
a[3][0]

0.0

In [122]:
a[0] = a[0] << 1 | 1
a

array([255], dtype=uint8)

In [84]:
2//2

1

In [67]:
3<<1

6